In [12]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [13]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch

In [14]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [15]:
PREFIX = 'jun19'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
sats_games

['jun19/jun19_2_no_activity',
 'jun19/jun19_3_undersell_allowed',
 'jun19/jun19_1_high_speed',
 'jun19/jun19_3_no_activity',
 'jun19/jun19_0_high_speed',
 'jun19/jun19_0_no_activity',
 'jun19/jun19_0_medium_speed',
 'jun19/jun19_1_no_activity',
 'jun19/jun19_2_undersell_allowed',
 'jun19/jun19_0_undersell_allowed',
 'jun19/jun19_4_base',
 'jun19/jun19_3_high_speed',
 'jun19/jun19_2_base',
 'jun19/jun19_3_base',
 'jun19/jun19_1_undersell_allowed',
 'jun19/jun19_0_base',
 'jun19/jun19_4_no_activity',
 'jun19/jun19_1_medium_speed',
 'jun19/jun19_2_high_speed',
 'jun19/jun19_1_base',
 'jun19/jun19_4_medium_speed',
 'jun19/jun19_4_high_speed',
 'jun19/jun19_4_undersell_allowed',
 'jun19/jun19_2_medium_speed',
 'jun19/jun19_3_medium_speed']

# For running the real dumb test case

In [16]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

25


In [21]:
num_iters = 10_000_000
time_limit_seconds = 3600 * 12
for config in pgrid:
    CONFIG = 'cfr_external' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 3_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun20external', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 1019638
Submitted batch job 1019639
Submitted batch job 1019640
Submitted batch job 1019641
Submitted batch job 1019642
Submitted batch job 1019643
Submitted batch job 1019644
Submitted batch job 1019645
Submitted batch job 1019646
Submitted batch job 1019647
Submitted batch job 1019648
Submitted batch job 1019649
Submitted batch job 1019650
Submitted batch job 1019651
Submitted batch job 1019652
Submitted batch job 1019654
Submitted batch job 1019655
Submitted batch job 1019656
Submitted batch job 1019657
Submitted batch job 1019658
Submitted batch job 1019659
Submitted batch job 1019661
Submitted batch job 1019662
Submitted batch job 1019663
Submitted batch job 1019664
Submitted batch job 1019665
Submitted batch job 1019666
Submitted batch job 1019667
Submitted batch job 1019668
Submitted batch job 1019669
Submitted batch job 1019670
Submitted batch job 1019671
Submitted batch job 1019672
Submitted batch job 1019673
Submitted batch job 1019674
Submitted batch job 

In [20]:
num_iters = 30_000_000
time_limit_seconds = 3600 * 24
for config in pgrid:
    CONFIG = 'cfr_outcome' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 3_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun20outcome', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 1019562
Submitted batch job 1019563
Submitted batch job 1019564
Submitted batch job 1019565
Submitted batch job 1019566
Submitted batch job 1019567
Submitted batch job 1019568
Submitted batch job 1019569
Submitted batch job 1019570
Submitted batch job 1019571
Submitted batch job 1019572
Submitted batch job 1019573
Submitted batch job 1019574
Submitted batch job 1019575
Submitted batch job 1019576
Submitted batch job 1019577
Submitted batch job 1019578
Submitted batch job 1019579
Submitted batch job 1019580
Submitted batch job 1019581
Submitted batch job 1019582
Submitted batch job 1019583
Submitted batch job 1019584
Submitted batch job 1019585
Submitted batch job 1019586
Submitted batch job 1019587
Submitted batch job 1019588
Submitted batch job 1019589
Submitted batch job 1019590
Submitted batch job 1019591
Submitted batch job 1019592
Submitted batch job 1019593
Submitted batch job 1019594
Submitted batch job 1019595
Submitted batch job 1019596
Submitted batch job 

In [ ]:
# for config in pgrid:
#     CONFIG = 'ppo_no_ent_jun14'
#     # CONFIG = 'ppo_may31_23' # The YML network config folder for the main jobs
#     # BR_CONFIG = 'ppo_may31_23' 
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br false --br_overrides "--total_timesteps 1_000_000"'        
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun14ent_3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)

In [10]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u newmanne --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [19]:
check_queue()

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
           1019561                  jun19_jun19_3_undersell_allowed-cfr_outcomemccfr_outcome-100_jun19outcome  R .      0:07      1 ada01
           1019560                        jun19_jun19_2_no_activity-cfr_outcomemccfr_outcome-100_jun19outcome  R .      0:10      1 ada01
           1019502                                                                                    jupyter  R .  14:56:25      1 ada21


In [ ]:
!squeue | wc -l

In [ ]:
!squeue | grep "may30" | wc -l

In [ ]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")